В этом уроке применим линейную регрессию на практике – попробуем предсказать стоимость машин и понять, от каких факторов зависит ценообразование на автомобили. Помимо этого узнаем, какие переменные важны для прогнозирования и насколько хорошо полученная модель описывает данные. 

# Задание:
1. Загрузите данные, проверьте правильность, наличие пропущенных значений, типы данных.
2. Создайте новый признак – марку автомобиля (company). Машины каких производителей встречаются в датасете? Далее исправьте названия и проверьте изменения.
3. Оставьте только часть предикторов, после чего посчитайте корреляцию между price и другими переменными.
4. Преобразуйте категориальные переменные с помощью pd.get_dummies(). 
5. Постройте модель с одним предиктором цены – horsepower. Какой процент изменчивости объясняет полученная модель? (\(R^2\))
6. Далее – две модели (со всеми предикторами и со всеми, кроме марок машин). Обратите внимание на изменения в \(R^2\), коэффициентах и их значимости. Какую модель лучше оставить? 
7. Заполните пропуски в результатах.

## 1. Загрузите данные, проверьте типы, пропущенные значения. 

Сколько пропущенных значений встретилось в датасете?

In [132]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats

In [3]:
df = pd.read_csv('data2/cars.csv')
df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [4]:
df.iloc[:,0:13].head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3


In [5]:
df.iloc[:,14:26].head()

,enginetype,cylindernumber,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [6]:
df.dtypes

car_ID                int64
symboling             int64
CarName              object
fueltype             object
aspiration           object
doornumber           object
carbody              object
drivewheel           object
enginelocation       object
wheelbase           float64
carlength           float64
carwidth            float64
carheight           float64
curbweight            int64
enginetype           object
cylindernumber       object
enginesize            int64
fuelsystem           object
boreratio           float64
stroke              float64
compressionratio    float64
horsepower            int64
peakrpm               int64
citympg               int64
highwaympg            int64
price               float64
dtype: object

In [7]:
df.isna().sum()

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

Ответ: `0`

## 2. Сгенерируем первый признак.

Использовать полное название машины – не самый хороший вариант, поэтому создадим новый признак – марку автомобиля (company). Для этого используйте столбец CarName, разбейте значения ячеек по пробелу и запишите в колонку первый элемент. Например:
```
'audi 100 ls' → 'audi'
```
Сколько всего уникальных марок машины встречается в датасете? Столбец CarName с полным названием машины удалите из датасета, а также car_ID, они не пригодятся для дальнейшего анализа.

In [8]:
df.CarName.nunique()

147

In [9]:
#df['company'] = df.CarName.str.split().str[0]
df.insert(0, 'company', value=df.CarName.str.split().str[0])
#.apply(lambda x: x.split(' ')[0])
df.company.unique()

array(['alfa-romero', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'maxda', 'mazda', 'buick', 'mercury',
       'mitsubishi', 'Nissan', 'nissan', 'peugeot', 'plymouth', 'porsche',
       'porcshce', 'renault', 'saab', 'subaru', 'toyota', 'toyouta',
       'vokswagen', 'volkswagen', 'vw', 'volvo'], dtype=object)

In [10]:
df.head(5)

,company,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,alfa-romero,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,alfa-romero,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,alfa-romero,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,audi,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,audi,5,2,audi 100ls,gas,std,four,sedan,4wd,front,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [109]:
df.company.nunique()

28

Ответ: `28`

## 2.2. Теперь внимательнее посмотрите на уникальные значения company. Часть из них оказалась с ошибками!
```
'maxda' → 'mazda'
'Nissan' → 'nissan'
'porcshce' → 'porsche'
'toyouta' → 'toyota'
'vokswagen' & 'vw' → 'volkswagen'
```
Сколько уникальных производителей осталось в итоге?

In [11]:
df.company = df.company.replace('alfa-romero','alfa-romeo')
df.company = df.company.replace('maxda','mazda')
df.company = df.company.replace('Nissan','nissan')
df.company = df.company.replace('porcshce','porsche')
df.company = df.company.replace('toyouta','toyota')
df.company = df.company.replace('vokswagen','volkswagen')
df.company = df.company.replace('vw','volkswagen')
df.company.unique()

array(['alfa-romeo', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'mazda', 'buick', 'mercury', 'mitsubishi',
       'nissan', 'peugeot', 'plymouth', 'porsche', 'renault', 'saab',
       'subaru', 'toyota', 'volkswagen', 'volvo'], dtype=object)

In [12]:
#в прошлом задании надо было 
df = df.drop(columns=['CarName','car_ID'])
df.head(5)

,company,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,alfa-romeo,3,gas,std,two,convertible,rwd,front,88.6,168.8,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,alfa-romeo,3,gas,std,two,convertible,rwd,front,88.6,168.8,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,alfa-romeo,1,gas,std,two,hatchback,rwd,front,94.5,171.2,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,audi,2,gas,std,four,sedan,fwd,front,99.8,176.6,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,audi,2,gas,std,four,sedan,4wd,front,99.4,176.6,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [13]:
df.company.nunique()

22

Ответ: `22`

## 3. Отлично! Чтобы не перегружать модель большим количеством предикторов, оставим только часть из них:
```
'company', 'fueltype', 'aspiration','carbody', 'drivewheel', 'wheelbase', 'carlength','carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower'
```
также не забыв про то, что мы предсказываем – 'price'. 

После этого посчитайте корреляцию между price и другими переменными. Чему равна корреляция между price и horsepower? Ответ округлите до 2 знаков после точки.

удаляем предикторы не из списка, price - ЗП

In [14]:
thelist = ['price','company', 'fueltype', 'aspiration','carbody', 'drivewheel', 'wheelbase', 'carlength','carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower']
#todelete = list(set(df.columns.to_list()) - set(thelist))
#df = df.drop(columns=todelete)
#я хочу помнить что я написал это глупое решение
df = df[thelist]
df.head()

,price,company,fueltype,aspiration,carbody,drivewheel,wheelbase,carlength,carwidth,curbweight,enginetype,cylindernumber,enginesize,boreratio,horsepower
0,13495.0,alfa-romeo,gas,std,convertible,rwd,88.6,168.8,64.1,2548,dohc,four,130,3.47,111
1,16500.0,alfa-romeo,gas,std,convertible,rwd,88.6,168.8,64.1,2548,dohc,four,130,3.47,111
2,16500.0,alfa-romeo,gas,std,hatchback,rwd,94.5,171.2,65.5,2823,ohcv,six,152,2.68,154
3,13950.0,audi,gas,std,sedan,fwd,99.8,176.6,66.2,2337,ohc,four,109,3.19,102
4,17450.0,audi,gas,std,sedan,4wd,99.4,176.6,66.4,2824,ohc,five,136,3.19,115


In [15]:
dfquant = df.dtypes[df.dtypes!=object].index.to_list()
dfquant
#только количественные, хотя я не проверил нету ли цифр типа str

['price',
 'wheelbase',
 'carlength',
 'carwidth',
 'curbweight',
 'enginesize',
 'boreratio',
 'horsepower']

In [16]:
df[dfquant].corr()


,price,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower
price,1.000000,0.577816,0.682920,0.759325,0.835305,0.874145,0.553173,0.808139
wheelbase,0.577816,1.000000,0.874587,0.795144,0.776386,0.569329,0.488750,0.353294
carlength,0.682920,0.874587,1.000000,0.841118,0.877728,0.683360,0.606454,0.552623
carwidth,0.759325,0.795144,0.841118,1.000000,0.867032,0.735433,0.559150,0.640732
curbweight,0.835305,0.776386,0.877728,0.867032,1.000000,0.850594,0.648480,0.750739
enginesize,0.874145,0.569329,0.683360,0.735433,0.850594,1.000000,0.583774,0.809769
boreratio,0.553173,0.488750,0.606454,0.559150,0.648480,0.583774,1.000000,0.573677
horsepower,0.808139,0.353294,0.552623,0.640732,0.750739,0.809769,0.573677,1.000000


In [17]:
df[dfquant].corr().price['horsepower'].round(2)

0.81

Ответ: `0.81`

## 4. Последний шаг в подготовке данных: линейная регрессия в python не справляется с категориальными переменными (типом object в pandas), поэтому давайте применим pd.get_dummies(). Пример использования:
```python
df_dummy = pd.get_dummies(data=cars[[список_столбцов_типа_object]], drop_first = True)
```
Не забудьте присоединить к полученному датасету столбцы с переменными других типов :) Также обратите внимание: мы должны использовать тот же набор колонок, который был на предыдущем шаге!

Сколько колонок теперь имеется в датасете?

In [30]:
df_dummy = pd.get_dummies(data=df.drop(columns=dfquant), drop_first = True)
df_dummy.head()

,company_audi,company_bmw,company_buick,company_chevrolet,company_dodge,company_honda,company_isuzu,company_jaguar,company_mazda,company_mercury,...,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,True,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,True,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,...,True,False,False,False,True,False,False,False,False,False


In [31]:
df2 = pd.concat([df_dummy, df[dfquant]], axis=1)
df2.head()

,company_audi,company_bmw,company_buick,company_chevrolet,company_dodge,company_honda,company_isuzu,company_jaguar,company_mazda,company_mercury,...,cylindernumber_twelve,cylindernumber_two,price,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower
0,False,False,False,False,False,False,False,False,False,False,...,False,False,13495.0,88.6,168.8,64.1,2548,130,3.47,111
1,False,False,False,False,False,False,False,False,False,False,...,False,False,16500.0,88.6,168.8,64.1,2548,130,3.47,111
2,False,False,False,False,False,False,False,False,False,False,...,False,False,16500.0,94.5,171.2,65.5,2823,152,2.68,154
3,True,False,False,False,False,False,False,False,False,False,...,False,False,13950.0,99.8,176.6,66.2,2337,109,3.19,102
4,True,False,False,False,False,False,False,False,False,False,...,False,False,17450.0,99.4,176.6,66.4,2824,136,3.19,115


Ответ: `49`

## 5. Сначала построим небольшую модель всего с одним предиктором цены (price) – horsepower.

Какой процент изменчивости объясняет полученная модель? (округлите до целого)

In [56]:
stats.linregress(df2.horsepower, df2.price)
#R**2 нету

LinregressResult(slope=163.2630609696478, intercept=-3721.7614943227454, rvalue=0.8081388225362213, pvalue=1.4834365732943386e-48, stderr=8.351478808257406, intercept_stderr=929.8492418134903)

In [57]:
res = smf.ols('price ~ horsepower + 1', df2).fit()
print(res.summary())
# horsepower+1 зачем?  кст в чужом решении +1 нету
#аналог add_constant
#X = sm.add_constant(X) # добавить константу, чтобы был свободный член 
#adds a column of constants (1-s)
#а что, если не добавить 1, не будет свободного члена? будет же
#This column of ones corresponds to x_0 in 
#the simple linear regression equation: y_hat = b_0 * x_0 + b_1 * x_1
#делаю без +1 - результат почти не отличается и свободный член не пропадает

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     382.2
Date:                Mon, 09 Oct 2023   Prob (F-statistic):           1.48e-48
Time:                        22:13:04   Log-Likelihood:                -2024.0
No. Observations:                 205   AIC:                             4052.
Df Residuals:                     203   BIC:                             4059.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3721.7615    929.849     -4.003      0.0

Ответ: 65% 

## 6. Теперь – две модели:

* модель со всеми предикторами
* модель со всеми предикторами, кроме марок машин

Обратите внимание на изменения в R^2 , коэффициентах и их значимости. Какую модель лучше оставить? 

In [83]:
cols = df2.columns.to_list()
cols.remove('price')
all_columns = " + ".join(cols)
all_columns
formula = "price ~ " + all_columns
res = smf.ols(formula, df2).fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     81.09
Date:                Mon, 09 Oct 2023   Prob (F-statistic):           4.86e-89
Time:                        22:31:52   Log-Likelihood:                -1804.2
No. Observations:                 205   AIC:                             3702.
Df Residuals:                     158   BIC:                             3858.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept     

In [107]:
# cols2 = pd.Series(df2.columns)
# cols2[cols2.str.startswith('company')].to_list()
compcols = df2.filter(like='company').columns.to_list()
compcols = set(compcols)
cols2 = [x for x in cols if x not in compcols]

all_columns = " + ".join(cols2)
formula = "price ~ " + all_columns
res = smf.ols(formula, df2).fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.901
Method:                 Least Squares   F-statistic:                     72.32
Date:                Mon, 09 Oct 2023   Prob (F-statistic):           9.86e-81
Time:                        22:57:49   Log-Likelihood:                -1881.6
No. Observations:                 205   AIC:                             3817.
Df Residuals:                     178   BIC:                             3907.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept     

In [ ]:
#часть марок не значима, adj. R**2 0.948->0.901 не сильно упал

Оставляем

* **Где меньше предикторов, ведь R2 изменился не очень сильно, а часть марок вообще не значима**
* Вообще с одним предиктором – horsepower!
* Где больше предикторов, там R2 самый большой

In [144]:
dfres = res.summary2().tables[1]
dfres.iloc[:, 2][dfres.iloc[:, 4]>0.05]
#захотелось конвертировать результат теста в датафрейм

enginetype_ohc[T.True]     4.464458
enginetype_ohcf[T.True]    3.905604
carwidth                   2.139010
horsepower                 5.193205
Name: t, dtype: float64

## 7. Заполните пропуски:

Выбранная модель объясняет примерно **90%** дисперсии (окр. до целого). Среди предикторов **23** из 27 оказались не значимыми (p > 0.05). Пример интерпретации: при единичном изменении показателя horsepower, цена **растет** на **86.8164** (без округления).